# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
import warnings
from constants import *
import math
import os
import random
import dill

# Helper Methods

In [2]:
def generate_random_sample(X, size):
    '''
    Given a list X, 
    generate random samples of given size
    '''
    Z_temp = random.sample(list(X), size)
    
    #Concatenation
    Z = [Z_temp[0]]
    for val in Z_temp[1:]:
        Z = np.concatenate([Z,[val]])
    
    return Z

def convert_values_to_list(list_val):
    '''
    Given a list X = [1 2 3] , 
    return X = [[1],[2],[3]]
    '''
    X = []
    for i in list_val:
        X.append([i])
    return X

def conversion_list_of_list(X, DIMENSION):
    '''
    Given a list X with values in lists,
    X = [[ 3  2  2],[4 8 10]]
    Convert each value to a list
    Return list of lists, array of lengths of each sequence
    X = [[3] [2] [2] [4] [8] [10]]
    ''' 
    X_new = []
    length = len(X)
    for idx, val_list in enumerate(X):
        Y = []
        for val in val_list:
            Y.append([val])
        X_new.append(Y)

    #Concatenation
    Z = X_new[0]
    for val_list in X_new[1:]:
        Z = np.concatenate([Z,val_list])

    # assign array of lengths for HMM
    lengths = [DIMENSION]*length
    
    return Z,lengths

#Calculate likelihood for given sequence according to given HMMs and return HMM
def likelihood_sequence(sequence, HMM_array):
    '''
    Given list of K HMMs and sequence,
    determines likelihood of sequence under all HMM models
    Returns index of HMM which has max likelihood
    ''' 
    scores = []
    length = [len(sequence)]
    for i, HMM in enumerate(HMM_array):
        calculated_score = HMM.score(sequence, length)
        scores.append(calculated_score)
    idx = scores.index(max(scores))
    return idx

def HMM_model_stats(model):
    '''
    Details of HMM model
    ''' 
    print("*************************************")
    print("Transition matrix")
    print(model.transmat_)
    print("*************************************")
    print("Means and stds of each hidden state")
    for i in range(model.n_components):
        print("Hidden state {0}".format(i))
        print("mean = ", model.means_[i])
        print("std = ", [np.sqrt(model.covars_[i])])
        print()

def BIC(model,X,lengths):
    LogLikelihood = model.score(X,lengths)
    num_hidden_states = model.n_components
    # D counts transition matrix (emission estimated by PDF), means = num_hidden_states  
    # covariance matrix = num_hidden_states
    D = num_hidden_states**2-num_hidden_states + 2*num_hidden_states
    BIC = LogLikelihood - (D/2)*np.log(len(X))
    return BIC
    
def BIC_array(HMM_array,X_i):
    BIC_total = 0
    for i in range(len(X_i)):
        model = HMM_array[i]
        X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        BIC_total+= BIC(model,X,lengths)
    return BIC_total

def likelihood_array(HMM_array,X_i):
    likelihood_total = 0
    for i in range(len(X_i)):
        if(len(X_i[i])>=HMM_array[i].n_components):
            model = HMM_array[i]
            X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
            LogLikelihood = model.score(X,lengths)
            likelihood = LogLikelihood 
            likelihood_total+= likelihood
    return likelihood_total

def plot_BIC(list_k, BIC_score):
    fig = plt.subplot(111)
    plt.plot(list_k, BIC_score, marker='o')  
    plt.xlabel('Value of K')
    plt.ylabel('Objective')
    plt.title('BIC')
    plt.show() 

def print_stats(assignments,length):
    for i in range(length):
        if(i%100==0):
            print(assignments[i])   

# Load Data and Clean

In [3]:
df_rnaseq = pd.read_csv('RPKMOutput/RNASeq.txt',sep=' ')

In [4]:
#Dataset
df_rnaseq = df_rnaseq[(df_rnaseq['cdReads0'] >= 10) & (df_rnaseq['cdReads1'] >= 10) & (df_rnaseq['cdReads2'] >= 10)& (df_rnaseq['cdReads3'] >= 10)& (df_rnaseq['cdReads4'] >= 10)]
df_main = df_rnaseq[['cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']]
LENGTH,DIMENSION = df_main.shape
print("Dataset size is",LENGTH)
print("Features are", DIMENSION)
print(df_main.head(5))
X = df_main.values
print("****************************")
print("First 5 log2 values\n",X[:5])

Dataset size is 10340
Features are 5
     cdRPKM0    cdRPKM1    cdRPKM2    cdRPKM3    cdRPKM4
0   6.642349   5.591876   6.004645   5.369988   8.507454
2   8.115351   9.879314   9.263817   7.164127   7.730539
3  54.937502  55.371390  64.913315  67.172940  78.293873
5  49.166094  35.735701  40.321640  35.461616  35.952554
8  35.030842  32.456656  35.944257  37.945878  41.572410
****************************
First 5 log2 values
 [[  6.64234888   5.59187561   6.00464467   5.36998757   8.50745417]
 [  8.11535125   9.87931428   9.2638167    7.16412693   7.73053887]
 [ 54.93750188  55.37139034  64.91331516  67.17294025  78.29387309]
 [ 49.16609435  35.73570142  40.32164016  35.46161645  35.95255399]
 [ 35.03084223  32.45665629  35.94425662  37.94587804  41.57241028]]


In [5]:
for idx,row in enumerate(X):
    temp = X[idx,0]
    X[idx,0]=0
    X[idx,1]-=temp
    X[idx,2]-=temp
    X[idx,3]-=temp
    X[idx,4]-=temp 
print("First 5 log2 values after normalizing\n",X[:5])

First 5 log2 values after normalizing
 [[  0.          -1.05047327  -0.63770421  -1.27236132   1.86510529]
 [  0.           1.76396303   1.14846545  -0.95122432  -0.38481238]
 [  0.           0.43388846   9.97581328  12.23543837  23.35637121]
 [  0.         -13.43039293  -8.8444542  -13.7044779  -13.21354036]
 [  0.          -2.57418594   0.91341439   2.91503582   6.54156806]]


# Arrays with HMM models for 1<=K<=20

In [6]:
HMM_K_ARRAYS = []
X_i_K_ARRAYS = []

# Check likelihood and do assignments

In [9]:
K_values = [5]

In [ ]:
for K in K_values:
    HMM_array = []
    X_i = []
    print("**************** K =", K ,"************************")
    for i in range(K):
        X_i.append([])
        
    NUM_ITERATIONS = 0
    NUM_CLUSTER_PREV = {}
    NUM_CLUSTER_NOW = {}
    
    # Sequences for initial HMM estimation
    # Make K subsets data of LENGTH
    for i in range(LENGTH):
        for j in range(K):
            if(i%K==j):
                X_i[j].append(list(X[i]))
                NUM_CLUSTER_PREV[i] = j
                
    for i in range(K):
        model = hmm.GaussianHMM(n_components=3,covariance_type='diag')
        X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        model.fit(X_temp, lengths)
        HMM_array.append(model)
    
    likelihood_prev = likelihood_array(HMM_array,X_i)
    print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_prev)
    NUM_ITERATIONS+=1
    while (True):
        # Assign all sequences to HMM models

        print("************ Check likelihood of sequence in HMM  *********")
        NUM_CLUSTER_NOW = {}
        for idx,x in enumerate(X):
            sequence = convert_values_to_list(x)
            hmm_index = likelihood_sequence(sequence, HMM_array)
            X_i[hmm_index].append(list(x))
            NUM_CLUSTER_NOW[idx] = hmm_index
        print("************ Checking likelihood done  *********")

        # Re-estimate parameters for new HMMs
        print("************ Re-estimating HMM *********")
        HMM_array_prev = HMM_array
        HMM_array = []
        for i in range(K):
            model = hmm.GaussianHMM(n_components=3,covariance_type='diag')
            if(len(X_i[i])>=model.n_components):
                X_temp, lengths = conversion_list_of_list(X_i[i], DIMENSION)
                model.fit(X_temp, lengths)
                HMM_array.append(model)
            else:
                HMM_array.append(HMM_array_prev[i])
        print("************ Re-estimation done *********")
        likelihood_curr = likelihood_array(HMM_array,X_i)
        print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_curr)
        print("*****************************************")

        # if no reassignments, then break
        if ((NUM_CLUSTER_PREV == NUM_CLUSTER_NOW)):
            HMM_K_ARRAYS.append(HMM_array)
            X_i_K_ARRAYS.append(X_i)
            break
        else:
            # initialize empty subsets of data for next iteration
            X_i = []
            for i in range(K):
                X_i.append([])

            NUM_CLUSTER_PREV = NUM_CLUSTER_NOW
            print("Num iterations is:", NUM_ITERATIONS)
            NUM_ITERATIONS += 1
            likelihood_prev = likelihood_curr
    print("**********************************************************\n\n")

In [11]:
print(len(HMM_K_ARRAYS))
print(len(X_i_K_ARRAYS))

1
1


In [20]:
import seaborn as sns

def plot_cluster(X,count):
    fig = plt.subplot(111)
    axes = plt.gca()
    axes.set_ylim([-500,500])
    var_plot_list = ['cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']
    total=0
    for i in range(len(X)):
        fig.plot(var_plot_list, X[i])
        total+=1
    title = "HMM "+ str(count)+" : " + str(total) + " points "
    plt.title(title)
    plt.savefig('/Users/akankshitadash/Desktop/Gene Ontology/5points/RNASeq Normalize/Clusters/Cluster'+str(count)+'.png')
    plt.show()
    
def plot_heatmap(X,idx):
    plt.figure()
    sns.heatmap(X,vmin=-500, vmax=500)
    plt.title('Heatmap'+str(idx))
    plt.savefig('/Users/akankshitadash/Desktop/Gene Ontology/5points/RNASeq Normalize/Heat Map/HeatMap'+str(idx+1)+'.png')
    plt.show()

In [ ]:
for idx,X in enumerate(X_i_K_ARRAYS[0]):
    plot_cluster(X,idx+1)
for idx,X in enumerate(X_i_K_ARRAYS[0]):
    plot_heatmap(X,idx)

In [34]:
df = pd.read_csv('RPKMOutput/RNASeq.txt',sep=' ')
df = df[(df['cdReads0'] >= 10) & (df['cdReads1'] >= 10) & (df['cdReads2'] >= 10)& (df['cdReads3'] >= 10)& (df['cdReads4'] >= 10)]
df = df.dropna()
df.head(5)

,AccNum,GeneName,cdReads0,cdRPKM0,cdReads1,cdRPKM1,cdReads2,cdRPKM2,cdReads3,cdRPKM3,cdReads4,cdRPKM4
0,NM_017847,ODR4,50.0,6.642349,62.0,5.591876,71.0,6.004645,30.0,5.369988,51.0,8.507454
2,NM_001003803,ATP5S,29.0,8.115351,52.0,9.879314,52.0,9.263817,19.0,7.164127,22.0,7.730539
3,NM_001003800,BICD2,778.0,54.937502,1155.0,55.371390,1444.0,64.913315,706.0,67.172940,883.0,78.293873
5,NM_016647,THEM6,170.0,49.166094,182.0,35.735701,219.0,40.321640,91.0,35.461616,99.0,35.952554
8,NM_016640,MRPS30,255.0,35.030842,348.0,32.456656,411.0,35.944257,205.0,37.945878,241.0,41.572410


In [35]:
df = df[['AccNum','GeneName','cdReads0','cdReads1','cdReads2','cdRPKM0','cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']]
df[['cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']] = df[['cdRPKM1','cdRPKM2','cdRPKM3','cdRPKM4']].sub(df['cdRPKM0'], axis=0)
df[['cdRPKM0']]=0
df.head(5)


,AccNum,GeneName,cdReads0,cdReads1,cdReads2,cdRPKM0,cdRPKM1,cdRPKM2,cdRPKM3,cdRPKM4
0,NM_017847,ODR4,50.0,62.0,71.0,0,-1.050473,-0.637704,-1.272361,1.865105
2,NM_001003803,ATP5S,29.0,52.0,52.0,0,1.763963,1.148465,-0.951224,-0.384812
3,NM_001003800,BICD2,778.0,1155.0,1444.0,0,0.433888,9.975813,12.235438,23.356371
5,NM_016647,THEM6,170.0,182.0,219.0,0,-13.430393,-8.844454,-13.704478,-13.213540
8,NM_016640,MRPS30,255.0,348.0,411.0,0,-2.574186,0.913414,2.915036,6.541568


In [36]:
for idx,x in enumerate(X_i_K_ARRAYS[0]):
    genes=[]
    acc_nums=[]
    for row in x:
        temp = (df.loc[(df['cdRPKM0'] == row[0])& (df['cdRPKM1']== row[1]) & (df['cdRPKM2']== row[2])])
        if(not temp.empty):
            genes.append(temp['GeneName'].values[0])
            acc_nums.append(temp['AccNum'].values[0])
    print(len(x),len(genes))
    with open('/Users/akankshitadash/Desktop/Gene Ontology/5points/RNASeq Normalize/Gene Names/Gene'+str(idx+1)+'.txt','w') as f:
        for gene in genes:
            f.write("%s\n" % gene)
    with open('/Users/akankshitadash/Desktop/Gene Ontology/5points/RNASeq Normalize/Gene Names/AccNum'+str(idx+1)+'.txt','w') as f:
        for acc_num in acc_nums:
            f.write("%s\n" % acc_num)

869 869
3226 3226
1462 1462
223 223
4560 4560


In [39]:
dill.dump_session('../Weights/HMM_GaussianHMM_5points_RNASeq_normalize.db')